Evaluating reference data

## Loading packages

In [ ]:
import os.path as osp, os
os.environ["CUDA_VISIBLE_DEVICES"] = ''
import torch
from torchmetrics import MeanAbsolutePercentageError
from torch.nn import HuberLoss
from torch_geometric.loader import DataLoader
import os.path as osp, pickle
from data.graphdataset import ThermoMLDataset, ramirez, ThermoMLpara
from train.parametrisation import  MAPE
from epcsaft import epcsaft_cython
import polars as pl
torch.cuda.is_available()

In [ ]:
device = torch.device("cpu")

In [ ]:
HLoss = HuberLoss("mean").to(device)
mape = MeanAbsolutePercentageError().to(device)

In [ ]:
pcsaft_den = epcsaft_cython.PCSAFT_den.apply
pcsaft_vp = epcsaft_cython.PCSAFT_vp.apply

path = osp.join("./data", "thermoml")
test_loader = ThermoMLDataset(path)

In [ ]:
def test(para_data):
    data = {"inchis":[],"mden":[],"mvp":[]} 
    for gh in test_loader:
        if gh.InChI not in para_data:
            continue
        parameters = para_data[gh.InChI][0]
        params = parameters.squeeze().to(torch.float64).numpy()
        rho = gh.rho.view(-1, 5).to(torch.float64).numpy()
        vp = gh.vp.view(-1, 5).to(torch.float64).numpy()
        mden, mvp = MAPE(params, rho, vp)
        data["inchis"].append(gh.InChI)
        data["mden"].append(mden)
        data["mvp"].append(mvp)
    return data

## Evaluating

In [ ]:
train_loader = ramirez('./data/ramirez2022')
para_data = {}
for graph in train_loader:
    para_data[graph.InChI] = [graph.para]
rapara = test(para_data)
rapara = pl.DataFrame(rapara)
rapara.write_csv('reframirez.csv')

In [ ]:
with open("./data/thermoml/raw/para3_fitted.pkl", "rb") as file:
    fitted_para = pickle.load(file)
data = {"inchis":[],"mden":[],"mvp":[]}
for inchi in fitted_para:
    data['inchis'].append(inchi)
    data['mden'].append(fitted_para[inchi][1])
    data['mvp'].append(fitted_para[inchi][2])
para = pl.DataFrame(data)
para.write_csv('refparametrisation.csv')

In [ ]:
train_loader = ThermoMLpara(path)
para_data = {}
for graph in train_loader:
    para_data[graph.InChI] = [graph.para]
tmlpara = test(para_data)
tmlpara = pl.DataFrame(tmlpara)
tmlpara.write_csv("./reftmlpara.csv")